# Dealing with Data Spring 2020 – Class 5

---

# Files and Printing

Often you will need to read data from a file, or write the output of a Python script back to a file. 

We use the `open` function to open the file in the appropriate mode, which takes two arguments: 

1. the name of the file,
2. and the mode. 

> `a_file = open(filename, mode)`

The `mode` is a single letter string that specifies if you are going to be reading from a file, writing to a file, or appending to the end of an existing file. The modes are: 

+ `'r'` : open a file for reading
+ `'w'` : open a file for writing (beware, this will overwrite any previously existing file) 
+ `'a'` : append (write to the end of a file) 

When reading a file, you usually want to iterate through the lines in that file using a `for loop`. Some other common methods for dealing with files are: 

+ `file.read()` : read the entire contents of a file into a string
+ `file.write(some_string)` : writes to the file (note, this doesn't automatically include new lines) 
+ `file.close()` : close the open file

# Writing a file to disk

# Reading a file from disk

Once we read the file, we have the lines in a big string. Let's process that big string a little bit:

^Please note that in Colab it takes a minute or so for the !rm changes to come into effect.

---

# ⭕ **QUESTIONS?**

---

# Python `os` standard library
Another addition to our file handling toolkit is the `os` library which provides ways to move files, make directories, and gather data about the file system. Like other standard libraries, we need to import it to use it via `import os`

^ Remember, all of the above is applying to our Colab environment. If you open up your computer's terminal or command line and type ```pwd``` (print working directory) you will most likely get a different response.

---

# ⭕ **QUESTIONS?**

---

# **Putting our Python to work: Exploring and Cleaning the Census Data file**

Our approach will be as follows:
* Inspect the data thoroughly to understand its benefits and risks for processing, and what information lies within
* Clean/fix the data to remove any issues that would prevent it from working with SQL, such as weird characters, too many columns, missing data, splitting values into multiple rows, or combining multiple values into a single row
* Structure the data found in the file as a Python native structure so we can manipulate and prepare it for use in SQL
* Migrate our approach to a dedicated Python file

We'll use: file read/write, loops, nested structures and UDFs to do all of this. (Some will be homework.)

---

## **1) Inspect our Data**

Upon visual inspection:
* there are 350+ columns
* there are zipcodes for just NYC 
* there are a mix of letter and numbers for values
* we have both percent and numbers values
* it appears to be comma-separated
* there are two column header lines: one with codes, and another with human-readable labels

Next - we inspect with Python in two ways: first we'll look at the raw file as a string, then we'll probe whether the comma-separation is fail safe or not.

To work in Google Colab, we must manually upload our CSV into the environment...

# **Review Comma-Based Separability**

In the first 500 characters, we see that the census has "code labels" for each column, none of which have funky characters that could trip us up with comma separation. 

We need to work with just that first line, though, so we'll use readline (which reads one entire line from the file) on the file handle

We knew it had 350+ and this is too many columns to effectively work with. 

Before we get to removing data, though, we need to put our data into a structure we can slice and dice. In other words, we need to transform our "string" data into a list of lists - a nested data structure where each row is a list, and within that list, each column is a list - a nested data structure.

For example: let's look at this simple 3x3 table:

## **2) Cleaning Up Wonky Data**

But before we can even create a nested structure - we need to be confident we can split the data correctly for every single line.

Unfortunately, CSV data is known to be particularly tricky because sometimes data sources use commas in column labels but surround those column headers with "" because Excel will treat it right. 

Python won't be so forgiving so we need to test for "" in the data - we'll do this using a for loop.

Uh-oh! those quotes spell trouble so now we need to see where that first quote appears and if a comma appears after it.

We suspect that those ""  in human-readable column headers of the second line of data are hiding "...**,**..." and will cause any nesting using comma-separation to create extra columns.

Let's prove it.

Now that we've proven those will be a problem, we are going to use Python to clean those up via our own User Defined Function for this purpose, because it may appear in another data source, too.

Before we create the UDF, let's describe what we want our function to do: 

1. it will accept a string input
2. it will remove " characters from the input
3. when it finds a , between "" it will replace , characters in the input with a `-`. However, it will NOT change `,` otherwise since it is a CSV file.
4. it will return a string output

And, we will create a test_input and expected_output to test our UDF during development:

It works! Now, let's fix our original input data string, `census_data`

---

## **3) Creating a Nested Structure**

Now that we've cleaned up the wonkiness in the data, we can create our nested structure using a `for` loop.

Let's start by reviewing our simple 3x3 table example:

In our example, the data is structured as:

`list[row][column]`

And we will use split commands to do the same for our data, this time creating another UDF.

Again - as before - let's state what it will do and create test data:
1. the UDF will take a data string as input
* it will create a list where each line in the data, identified using `\n`, is an item
* for each item in the first list `list[row]`, it will create a list of columns, using comma-separation (`,`) to identify each item
* it would be nice for the UDF user to specify what character separates rows and columns, separately
* the UDF will return the nested data structure

Now we can create a nested structure of our census data:

---

## **4) Moving from Notebooks to \*.py Files** 

But before you go to do that, we need to start moving the findings of our exploration into a dedicated python file for cleaning the Census data. 

We're making this migration because notebooks are great for exploring data, but as our files and project grow larger, it is simpler to run the Python files outside notebooks AND sometimes very large files can cause our notebooks to crash.

Here's what that new Python file needs to do:

1. read the census data file into a variable
2. clean the data by removing the "...,..." problem using a UDF
3. create a nested data structure
4. remove unwanted columns
5. create a new string from the nested structure
6. write the file to disk

We've already #1, #2, #3, and #6 together, so we'll isolate those below along with comments to do the other work before putting in its own file.

And recall that our UDFs has to be put ahead of the main program to work because the Main Program needs to know what happens in those UDFs.

Below is the exact code that will be put into its own file named `clean_census.py`.

```python
################################################################
##### UNQUOTABLE #############################################
################################################################

def unquotable(input_string): # removes pesky , within "" values before changing , to \t in program
    
    # remove " char by splitting the input string into a list using that char
    tmp_list = input_string.split('"')
 
    # remove , char
    for i in range(len(tmp_list)):
        # skip items that start/end with commas as these aren't quoted items
        if (tmp_list[i][0] == ",") or (tmp_list[i][-1] == ","):
            continue
        # replace , with - in quoted items
        else:
            tmp_list[i] = tmp_list[i].replace(",","-")

    # rejoin items into a string
    output = ",".join(tmp_list)

    # get rid of any ",," and ",,," as it will add extra columns
    output = output.replace(",,,",",")
    output = output.replace(",,",",")
    
    # return string
    return output

################################################################
##### NESTER #############################################
################################################################

def nester(input_string,input_row_delim,input_col_delim): # take a dataset string where each row is separated by input_row_delim and each column is separate by input_col_delim to create a nested object of lists

    # create a list item for each row in the file using the row delimiter
    row_list = input_string.split(input_row_delim)

    #output var
    nested_data = []
    
    # created nested structure to store each column separately
    # list of rows where each row is a list of columns)
    for i in range(len(row_list)): 
        row = row_list[i]
        col = row.split(input_col_delim)
        nested_data.append(col)
    
    # return the nested structure
    return nested_data


################################################################
##### MAIN PROGRAM #############################################
################################################################

# 1. read the census data file into a variable

file_handle = open('raw_census_2010_copy.csv',"r")
census_data = file_handle.read()
file_handle.close()

# 2. clean the data by removing the "...,..." problem using a UDF

clean_census_data = unquotable(census_data)

# 3. create a nested data structure with a UDF

nested_census_data = nester(clean_census_data,"\n",",")

# 4. remove unwanted columns using a UDF

# HOMEWORK

# 5. create a new string from the nested structure using a UDF

# HOMEWORK

# 6. write the file to disk

file_handle = open('raw_census_2010_copy.csv',"r")
file_handle.write('final data var from step #5 goes here')
file_handle.close()
```

---

# ⭕ **QUESTIONS?**

---